# 4.11. 添加技术指标
之前我提到过indicators（技术指标），下一步就该添加他们了，要做的肯定比“三连跌”这种复杂点。借用PyAlgoTrade这个框架的一个使用移动平均线的例子：

+ 收盘价高于平均价的时候，以市价买入
+ 持有仓位的时候，如果收盘价低于平均价，卖出
+ 只有一个待执行的订单

大多数代码不用改变，在 init 方法中加入移动平均线的实例化:
```
# 加入移动平均线
self.sma = bt.indicators.MovingAverageSimple(self.datas[0], period=self.params.maperiod)
```
当然买入卖出的逻辑依赖平均价，具体代码如下。

注意： 起始金额为1000元，无手续费，这和 PyAlgoTrade 保持一致。

In [14]:
#查询存储位置, 绝对路径的规范写法

import os
# print(os.path.abspath('.')) # 得到当前文件所在目录的绝对路径
# print(os.path.abspath(r'..')) # 得到当前文件所在目录的上一级目录的绝对路径
# print(os.path.abspath(r'../..')) # 得到当前文件所在目录的上一级目录的绝对路径


In [15]:
import datetime  #
import os.path  # 路径管理
import sys  # 获取当前运行脚本的路径 (in argv[0])

#导入backtrader框架
import backtrader as bt

# 创建策略继承bt.Strategy
class TestStrategy(bt.Strategy):
    params = (
        # 均线参数设置15天，15日均线
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        # 记录策略的执行日志
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # 保存收盘价的引用
        self.dataclose = self.datas[0].close
        print('切片访问前3个值', self.datas[0].array[:3])
        # 跟踪挂单
        self.order = None
        # 买入价格和手续费
        self.buyprice = None
        self.buycomm = None
        # 加入均线指标
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod)



    # 订单状态通知，买入卖出都是下单
    def notify_order(self, order):
        print('切片访问前3个值', self.sma.get(ago=0, size=3))
        if order.status in [order.Submitted, order.Accepted]:
            # broker 提交/接受了，买/卖订单则什么都不做
            return

        # 检查一个订单是否完成
        # 注意: 当资金不足时，broker会拒绝订单
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    '已买入, 价格: %.2f, 费用: %.2f, 佣金 %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                self.log('已卖出, 价格: %.2f, 费用: %.2f, 佣金 %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
            # 记录当前交易数量
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('订单取消/保证金不足/拒绝')

        # 其他状态记录为：无挂起订单
        self.order = None

    # 交易状态通知，一买一卖算交易
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('交易利润, 毛利润 %.2f, 净利润 %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # 记录收盘价
        self.log('Close, %.2f' % self.dataclose[0])

        # 如果有订单正在挂起，不操作
        if self.order:
            return

        # 如果没有持仓则买入
        if not self.position:
            # 今天的收盘价在均线价格之上
            if self.dataclose[0] > self.sma[0]:
                # 买入
                self.log('买入单, %.2f' % self.dataclose[0])
                    # 跟踪订单避免重复
                self.order = self.buy()
        else:
            # 如果已经持仓，收盘价在均线价格之下
            if self.dataclose[0] < self.sma[0]:
                # 全部卖出
                self.log('卖出单, %.2f' % self.dataclose[0])
                # 跟踪订单避免重复
                self.order = self.sell()



if __name__ == '__main__':
    # 创建Cerebro引擎
    cerebro = bt.Cerebro()
    # Cerebro引擎在后台创建broker(经纪人)，系统默认资金量为10000

    # 为Cerebro引擎添加策略
    cerebro.addstrategy(TestStrategy)

    # 获取当前运行脚本所在目录
    modpath = os.path.abspath(r'.')
    # 拼接加载路径
    datapath = os.path.join(modpath, '..\..\datas\orcl-1995-2014.txt')
    # 创建交易数据集
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # 数据必须大于fromdate
        fromdate=datetime.datetime(2000, 1, 1),
        # 数据必须小于todate
        todate=datetime.datetime(2000, 12, 31),
        reverse=False)

    # 加载交易数据
    cerebro.adddata(data)


    # 设置投资金额1000.0
    cerebro.broker.setcash(1000.0)

    # 每笔交易使用固定交易量
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    # 设置佣金为0.0
    cerebro.broker.setcommission(commission=0.0)

    # 引擎运行前打印期出资金
    print('组合期初资金: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    # 引擎运行后打期末资金
    print('组合期末资金: %.2f' % cerebro.broker.getvalue())

组合期初资金: 1000.00
切片访问前3个值 array('d', [26.27, 23.95, 22.68])
2000-01-24, Close, 24.10
2000-01-25, Close, 25.10
2000-01-25, 买入单, 25.10
切片访问前3个值 array('d', [24.38133333333333, 24.303333333333335, 24.339333333333332])
切片访问前3个值 array('d', [24.38133333333333, 24.303333333333335, 24.339333333333332])
切片访问前3个值 array('d', [24.38133333333333, 24.303333333333335, 24.339333333333332])
2000-01-26, 已买入, 价格: 25.24, 费用: 252.40, 佣金 0.00
2000-01-26, Close, 24.49
2000-01-27, Close, 23.04
2000-01-27, 卖出单, 23.04
切片访问前3个值 array('d', [24.339333333333332, 24.363333333333333, 24.34466666666667])
切片访问前3个值 array('d', [24.339333333333332, 24.363333333333333, 24.34466666666667])
切片访问前3个值 array('d', [24.339333333333332, 24.363333333333333, 24.34466666666667])
2000-01-28, 已卖出, 价格: 22.90, 费用: 252.40, 佣金 0.00
2000-01-28, 交易利润, 毛利润 -23.40, 净利润 -23.40
2000-01-28, Close, 21.07
2000-01-31, Close, 22.22
2000-02-01, Close, 24.02
2000-02-02, Close, 24.16
2000-02-02, 买入单, 24.16
切片访问前3个值 array('d', [24.17866666666667, 24.123333

让我们仔细地看一下出现在下面日志中的第一条记录： 不再是新千年的第一个交易日2000-01-03了。 变成了2000-01-24，怎么回事呢？ 是因为，框架根据新代码做出了改变： 在策略中我们加入了移动平均技术指标。 移动平均需要有个均线周期参数，程序根据这个参数回看计算前边的X条价格数据然后进行开仓判断，例子中周期是15。 2000-01-24就是第15天 backtrader 框架假定策略加入这个技术指标是有正当理由的，比如做开平仓的决策。 框架不会在数据没到位的时候就进行下一步。 在技术指标产生第一条数据之后，next方法第一个被调用。 在示例中只有一个技术指标，其实策略支持添加多个技术指标。
